In [1]:
# Importing Libraries

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

# Graphing libraries 
import matplotlib.pyplot as plt
import seaborn as sns

# Importing all soup to nuts libraries
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor
from sklearn.pipeline import Pipeline, make_pipeline

